In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/properati_final.csv',index_col=0,error_bad_lines=False)

<IPython.core.display.Javascript object>

In [15]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

In [16]:
cant_buckets = 100

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
#properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

#properati['price_range']=pd.cut(properati['price_per_m2'],rango)

#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 
properati.groupby('categories_by_price').count().sort_values('price')

,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,fecha,superficie_descubierta
categories_by_price,,,,,,,,,,,,,,,,,,,,,
78.0,519,519,519,519,371,371,519,519,519,519,...,519,519,76,311,136,519,519,519,519,397
24.0,718,718,718,718,480,480,718,718,718,718,...,718,718,16,307,49,718,718,718,718,517
47.0,824,824,824,824,604,604,824,824,824,824,...,824,824,123,404,172,824,824,824,824,521
37.0,1127,1127,1127,1127,771,771,1127,1127,1127,1127,...,1127,1127,82,503,170,1127,1127,1127,1127,731
73.0,1161,1161,1161,1161,910,910,1161,1161,1161,1161,...,1161,1161,208,695,303,1161,1161,1161,1161,831
49.0,1235,1235,1235,1235,929,929,1235,1235,1235,1235,...,1235,1235,174,646,295,1235,1235,1235,1235,764
32.0,1241,1241,1241,1241,837,837,1241,1241,1241,1241,...,1241,1241,77,570,145,1241,1241,1241,1241,806
51.0,1249,1249,1249,1249,912,912,1249,1249,1249,1249,...,1249,1249,192,635,292,1249,1249,1249,1249,743
16.0,1257,1257,1257,1257,672,672,1257,1257,1257,1257,...,1257,1257,13,423,31,1257,1257,1257,1257,1101


In [17]:
properati.loc[properati['categories_by_price']!= properati['categories_by_price']].sort_values('price') #veo si clasifico una a categoria Nan

,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,fecha,superficie_descubierta,categories_by_price
created_on,,,,,,,,,,,,,,,,,,,,,


In [18]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],properati['place_name'],properati['property_type'])
y = properati['categories_by_price']

### Random Search

In [19]:
%%notify

from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_jobs=-1)

max_features =["auto","sqrt","log2"]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"max_features":max_features,"n_estimators":n_estimators,}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=10,n_iter=3) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 329.11 segundos para 3 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.489 (std: 0.000)
Promedio validation score: 0.165 (std: 0.003)
Promedio fit time: 2.281s
Hiper-parametros: {'n_estimators': 40, 'max_features': 'auto'}

Puesto: 2
Promedio training score: 0.489 (std: 0.001)
Promedio validation score: 0.165 (std: 0.003)
Promedio fit time: 4.937s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'log2'}

Puesto: 3
Promedio training score: 0.490 (std: 0.000)
Promedio validation score: 0.164 (std: 0.004)
Promedio fit time: 4.411s
Hiper-parametros: {'n_estimators': 80, 'max_features': 'log2'}



<IPython.core.display.Javascript object>

In [20]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.16764447051921386